In [ ]:
!pip install cryptography

import os
import sys
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

# Encrypt data before uploading

TVB uses the RSA algorithm for asymmetric-key encryption and decryption of data. Unfortunately, RSA by itself is not enough because it can only encrypt limited amounts of data. 

To solve this problem, the data is encrypted using a symmetric-key encryption algorithm, namely AES, and the symmetric key is then encrypted using the public key of the RSA algorithm.

First, we read the public key,

In [ ]:
  path_to_public_key = 'public_key.pem' # enter path to public key here

  with open(path_to_public_key, "rb") as key_file:
    public_key = serialization.load_pem_public_key(
      key_file.read(),
      backend=default_backend()
    )

then randomly generate the AES symmetric-key and the initialization vector 

In [ ]:
symmetric_key = os.urandom(32)
iv = os.urandom(16)

and use them to initialize the AES cypher.

In [ ]:
cipher = Cipher(algorithms.AES(symmetric_key), modes.CTR(iv), backend=default_backend())
encryptor = cipher.encryptor()

The next step is to read the content from the file to be encrypted as binary and use the AES cypher to encrypt the content.

The encrypted content is saved by default at the location of the original data.

If we wish to upload data which requires more than one file (for example, Connectivity CSV), then we have to use the same AES key and initialization vector, so we'll have to:
1. reinitialize the AES cypher (rerun the previous code block)
2. change the value of the *path_to_file* parameter and rerun this next code block as well

In [ ]:
path_to_file = 'connectivity_76.zip' # enter path to data file here

with open(path_to_file, 'rb') as f:
  content = f.read()

encrypted_content = encryptor.update(content) + encryptor.finalize()

start_extension = path_to_file.rfind('.')
path_to_encrypt = path_to_file[:start_extension]
extension = path_to_file[start_extension:]

path_to_encrypt = path_to_encrypt + '_encrypted' + extension

with open(path_to_encrypt, 'wb') as f:
  f.write(encrypted_content)

The next step is to encrypt the AES symmetric key with the asymmetric public key obtained from TVB.

In [ ]:
encrypted_symmetric_key = public_key.encrypt(
  symmetric_key,
  padding.OAEP(
    mgf=padding.MGF1(algorithm=hashes.SHA256()),
    algorithm=hashes.SHA256(),
    label=None
  )
)

Lastly, we append the initialization vector to the end of the encrypted AES symmetric key and thus we save them both in the same file at a specified location.

In [ ]:
extended_encrypted_symmetric_key = encrypted_symmetric_key + iv
path_to_encrypted_key = './' # enter path for saving encrypted key here

with open(os.path.join(path_to_encrypted_key, 'encrypted_symmetric_key.pem'), 'wb') as f:
  f.write(extended_encrypted_symmetric_key)